In [50]:
import pandas as pd 
import numpy as np 
import networkx as nx 
import tqdm
import json 
import pickle
import matplotlib.pyplot as plt 
import seaborn as sns
from datetime import datetime
import dynetx as dn

In [3]:
G = nx.read_graphml('../data/clean/byuser.graphml',node_type=int)
print(len(G),len(G.edges))
for node in list(G.nodes)[:3]: print(node,G.nodes[node])
for node in list(G.edges)[:3]: print(node,G.edges[node])

8258 61366
0 {'province': 'PI', 'covidMeasure': 0, 'latitude': 43.85180773789867, 'longitude': 10.506207}
15775 {'province': 'LU', 'covidMeasure': 0, 'latitude': 43.85135, 'longitude': 10.506207}
1979 {'province': 'FI', 'covidMeasure': 0, 'latitude': 43.76973, 'longitude': 11.26562}
(0, 15775, 0) {'start': 414, 'end': 414, 'user': 'RobertaC2372', 'days': 0, 'geodistance': 0.0004577378986709846}
(0, 15775, 1) {'start': 414, 'end': 414, 'user': 'ClaudioC138', 'days': 0, 'geodistance': 0.0004577378986709846}
(0, 15775, 2) {'start': 414, 'end': 414, 'user': 'juan pablo A 116887066', 'days': 0, 'geodistance': 0.0004577378986709846}


# Dynetx

In [4]:
g = dn.DynGraph(edge_removal=True)

In [5]:
for i,j,k in G.edges:
    try : g.add_interaction(u=i, v=j, t=G.edges[(i,j,k)]['end'])
    except: continue

In [6]:
len([i for i in g.nodes()])

8258

In [7]:
dn.inter_event_time_distribution(g, u=0, v=15775)

{0: 5}

In [8]:
dn.algorithms.paths.time_respecting_paths(g,u=0)

[[(0, 12564, 197)], [(0, 14479, 209)], [(0, 1979, 237)], [(0, 15775, 414)]]

In [58]:
dn.algorithms.paths.time_respecting_paths(g,u=12564)

[[(12564, 0, 197)], [(12564, 15775, 197)]]

In [10]:
allPaths = dn.all_time_respecting_paths(g)

100%|████████████████████████████████████████████████████████████████████████████| 8258/8258 [3:26:18<00:00,  1.50s/it]


In [30]:
allPaths[(14479, 15775)]

[[(14479, 3293, 132)],
 [(14479, 4180, 132)],
 [(14479, 14911, 132)],
 [(14479, 14911, 132), (14911, 14411, 133)],
 [(14479, 14911, 132), (14911, 14854, 133)],
 [(14479, 14911, 132), (14911, 15006, 133)],
 [(14479, 7151, 140)],
 [(14479, 15548, 144)],
 [(14479, 502, 145)],
 [(14479, 14004, 153)],
 [(14479, 14382, 157)],
 [(14479, 14472, 157)],
 [(14479, 14415, 159)],
 [(14479, 14415, 159), (14415, 14484, 160)],
 [(14479, 14439, 161)],
 [(14479, 15635, 163)],
 [(14479, 719, 165)],
 [(14479, 638, 165)],
 [(14479, 9604, 170)],
 [(14479, 14397, 173)],
 [(14479, 14397, 173), (14397, 403, 174)],
 [(14479, 14665, 174)],
 [(14479, 1987, 174)],
 [(14479, 1987, 174), (1987, 2537, 175)],
 [(14479, 1987, 174), (1987, 1783, 176)],
 [(14479, 1987, 174), (1987, 2537, 175), (2537, 11914, 176)],
 [(14479, 1987, 174), (1987, 2192, 177)],
 [(14479, 1987, 174), (1987, 2165, 178)],
 [(14479, 1987, 174), (1987, 2192, 177), (2192, 1931, 178)],
 [(14479, 1987, 174), (1987, 2192, 177), (2192, 2592, 178)],
 [(1

In [20]:
len(allPaths)

295212

In [44]:
list(allPaths.keys())[:20]

[(0, 12564),
 (0, 14479),
 (0, 1979),
 (0, 15775),
 (15775, 14805),
 (15775, 12564),
 (15775, 14479),
 (15775, 1979),
 (15775, 0),
 (1979, 2154),
 (1979, 2373),
 (1979, 2184),
 (1979, 2160),
 (1979, 1981),
 (1979, 13659),
 (1979, 3210),
 (1979, 13656),
 (1979, 13832),
 (1979, 4085),
 (1979, 13010)]

In [25]:
with open('../data/clean/allPaths.PICKLE','wb') as jf:
    pickle.dump(allPaths,jf)

In [45]:
ap = dn.annotate_paths(allPaths[(1979, 2160)])

In [59]:
late = list()
slow = list()
long = list()
latest = 0
slowest = 0
longest = 0


for e, paths in tqdm.tqdm(allPaths.items()):
    
    stats = dn.annotate_paths(paths)
    shortest = len(stats['shortest'][0])
    fastest = len(stats['fastest'][0])
    foremost = len(stats['foremost'][0])
    
    if shortest == longest : long.append(e)
    elif shortest > longest :
        longest = shortest
        long = [e]
        
    if fastest == slowest : long.append(e)
    elif fastest > slowest :
        slowest = fastest
        slow = [e]
        
    if foremost == latest: long.append(e)
    elif foremost > latest :
        latest = foremost
        late = [e]
        
    
    
    

100%|█████████████████████████████████████████████████████████████████████████| 295212/295212 [09:40<00:00, 508.47it/s]


In [68]:
counter = 0
for path in allPaths[(1979, 1981)]:
    counter +=1
    if path[-1][1] not in  dn.all_neighbors(g,1981):
        
        print(path[-1][1],' ops')
        
print(counter, len(allPaths[(1979, 1981)]))

2154  ops
2373  ops
2160  ops
13659  ops
3210  ops
13656  ops
13832  ops
4085  ops
13010  ops
13120  ops
1878  ops
2350  ops
1772  ops
2584  ops
11360  ops
1991  ops
2429  ops
2447  ops
10676  ops
2006  ops
11360  ops
2162  ops
2374  ops
1898  ops
1498  ops
2468  ops
2401  ops
1976  ops
0  ops
15775  ops
37 37


In [61]:
allPaths[(1979, 1981)]

[[(1979, 2154, 110)],
 [(1979, 2373, 112)],
 [(1979, 2184, 136)],
 [(1979, 2160, 136)],
 [(1979, 1981, 136)],
 [(1979, 2184, 136), (2184, 13659, 137)],
 [(1979, 2184, 136), (2184, 3210, 138)],
 [(1979, 2184, 136), (2184, 13659, 137), (13659, 13656, 138)],
 [(1979, 2184, 136), (2184, 13659, 137), (13659, 13832, 138)],
 [(1979, 2184, 136), (2184, 4085, 139)],
 [(1979, 2184, 136),
  (2184, 13659, 137),
  (13659, 13832, 138),
  (13832, 13010, 139)],
 [(1979, 2184, 136),
  (2184, 13659, 137),
  (13659, 13832, 138),
  (13832, 13120, 139)],
 [(1979, 2322, 149)],
 [(1979, 1878, 155)],
 [(1979, 1878, 155), (1878, 2350, 156)],
 [(1979, 1878, 155), (1878, 1772, 156)],
 [(1979, 2584, 156)],
 [(1979, 2584, 156), (2584, 11360, 157)],
 [(1979, 2016, 162)],
 [(1979, 2016, 162), (2016, 1991, 163)],
 [(1979, 2016, 162), (2016, 2429, 164)],
 [(1979, 2016, 162), (2016, 2447, 164)],
 [(1979, 2016, 162), (2016, 10676, 164)],
 [(1979, 2016, 162), (2016, 2628, 165)],
 [(1979, 2016, 162), (2016, 2447, 164), (2

In [60]:
print(slowest,longest,latest)

1 1 1


In [56]:
len(long)

295212